In [89]:
from sys import platform
import os
import cv2
import json
from pupil_apriltags import Detector
import open3d as o3d
from glob import glob
import numpy as np
from scipy.spatial.transform import Rotation as R
import collections
import k3d
from PoseEstimation.Mesh_Matcher import *
from PoseEstimation.Colmap_Reader import *
from PoseEstimation.Apriltag_Colmap import *
from PoseEstimation.GT_Extration import rigid_transform_3D
import trimesh
import pyrender
from PIL import Image

In [3]:
a2d2 = [[-3.738231 ,-0.731755 ,-0.328742],
    [-2.637552 ,-0.291120 ,-2.638498],
    [0.646090 ,-0.735447 ,-0.999240],
    [-0.068454 ,-1.510249 ,2.817723],
    [-3.638991 ,-0.949203 ,1.054112],
    [-1.450608 ,-0.856347 ,0.655611]]

a2d1 = [[-0.463405 ,-0.737901 ,3.250370],
       [-2.756239 ,-0.312765 ,2.103297],
       [-1.103867 ,-0.729082 ,-1.151607],
       [2.675915 ,-1.559122 ,-0.357202],
       [0.901114 ,-0.947988 ,3.181215],
       [0.618660 ,-0.928016 ,0.953832]]

a1d1 = [[3.176764, -0.743093 ,0.347837],
       [1.207308 ,-0.537581 ,3.230441],
       [-1.180794, -0.635370 ,1.481291],
       [-1.357942, -1.557935 ,-0.596481],
       [1.503481 ,-1.525294, -3.660501],
       [0.554387 ,-0.842245 ,-0.227942]]

a1d2 = [[-2.751245, -0.733777 ,-0.333212],
       [-0.531045 ,-0.543927 ,-2.986102],
       [1.695879 ,-0.656546 ,-0.978193],
       [1.536978 ,-1.557403 ,1.012150],
       [-1.503375 ,-1.568758 ,3.784639],
       [-0.193254 ,-0.846000 ,0.419316]]

In [4]:
from math import pi
def position_error(T_e, T_gt):

    return np.linalg.norm(T_e- T_gt)


def rotation_error(R_e, R_gt):
    
    return (180/pi) * np.arccos((np.trace(np.matmul(R_e, R_gt.T))-1)/2)

def draw_registration_result(source, target, transformation, colored = True):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if colored:
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [5]:
def k3d_frustrum(pose, name, size=0.009, color=0x0000ff):
    # i.e. not using image sizes 
    pos = pose[:3, 3]
    
    forward = -pose[:3, 2] * size * -1.4
    right = -pose[:3, 0] * size * 1.25
    up = -pose[:3, 1] * size
    
    #verts = [pos, pos + forward*size ]
    verts = [pos, pos + forward - right*0.5 + up*0.5, pos + forward + right * 0.5 + up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 - up*0.5, pos + forward + right * 0.5 - up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 + up*0.5, pos + forward - right * 0.5 - up * 0.5, pos]
    verts += [pos, pos + forward + right*0.5 + up*0.5, pos + forward + right * 0.5 - up * 0.5, pos]
    return k3d.line(verts, color=color, width=2.5, name = name, shader="simple")

def visualization_registration(reg, mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffcc00, 0xccff00, 0x00ccff]

    for i, frame in enumerate(reg.keys()):
        pos = reg[frame][:3, 3] 
        vertices += pos.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame), size=0.1, color=colors[i % len(colors)]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    #plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()


def visualization_PI_IPhone(gt, reg,  mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00]

    for frame in reg.keys():
        est_pose = reg[frame][:3, 3]
        gt_pose = gt[frame][:3, 3]
 
        vertices += est_pose.tolist()
        vertices += gt_pose.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame) + "_GT", size=0.1, color=colors[0]) )
        frustrums.append( k3d_frustrum(gt[frame], name = str(frame)+ "Est", size=0.1, color=colors[1]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    #plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()


at_detector = Detector(
            families="tagStandard41h12",
            nthreads=1,
            quad_decimate=1.0,
            quad_sigma=0.0,
            refine_edges=1,
            decode_sharpening=0.25,
            debug=0,
        )


Register = collections.namedtuple(
    "RegisterInfo", ["CameraPose", "Intrinsic", "TagPose"])


In [6]:
if platform == "linux" or platform == "linux2":  
# linux
    dataset_path = r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1/images_undistorted"
    database_path = r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1/colmap_sparser"
    mesh_fullpath = r"/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1/data3d/textured_output.obj"
    #keypoints_gt = np.array(a1d2)
elif platform == "win32":
# Windows...
    # dataset_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1\frames"
    # database_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1\colmap_reconstruction"

    dataset_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1\images_undistorted"
    database_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1\colmap_sparser"
    mesh_fullpath = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1\data3d\textured_output.obj"
    
    
VISUALIZATION_3D = True

In [7]:
database_colmap = ColmapReader(database_path)
cameras, images, points3D = database_colmap.read_sparse_model()
pcd_reconstruction, _= database_colmap.read_dense_model()

Detected model format: '.bin'
Detected model format: '.bin'


In [8]:
registered_3d_points_xyz = {} # register for each tag (including only xyz)
registered_3d_points_info = {} # register for each tag (including other info)
registered_3d_points_repro_error = {} # register for each tag (including other info)
visited_id = []
for frame in images.values():

    img_fullpath = os.path.join(dataset_path, frame.name)
    img = cv2.imread(img_fullpath, cv2.IMREAD_GRAYSCALE)
    tags = at_detector.detect(img)
    if len(tags) == 0:
        continue

    tag_ids = [tag.tag_id for tag in tags]
    masks = get_mask(img_grayscale=img, tags=tags, visualization=False)

    # Find the corresponding 3d points
    search_space_pixels = np.array(frame.xys.astype(int))
    points_3d = frame.point3D_ids
    corresponding_3d_ids, _ = get_corresponding_3d_points_ids(masks, search_space_pixels, points_3d)

    # filter the duplicated ids
    for (tag_id, corresponding_3d_id) in zip(tag_ids, corresponding_3d_ids):
        non_visited_3d_id = filter_array(corresponding_3d_id, visited_id)
        non_visited_3d_id = list(non_visited_3d_id)
        visited_id.extend(non_visited_3d_id)

        corresponding_3d_xyz, corresponding_3d_repro_error, corresponding_3d_info = get_valid_3d_points(non_visited_3d_id, points3D)

        if len(corresponding_3d_xyz) == 0:
            continue
        if tag_id not in registered_3d_points_xyz.keys():
            registered_3d_points_xyz[tag_id] = corresponding_3d_xyz
            registered_3d_points_info[tag_id] = corresponding_3d_info
            registered_3d_points_repro_error[tag_id] = corresponding_3d_repro_error

        else:
            registered_3d_points_xyz[tag_id] = np.concatenate((registered_3d_points_xyz[tag_id], corresponding_3d_xyz), axis=0)
            registered_3d_points_info[tag_id] = np.concatenate((registered_3d_points_info[tag_id], corresponding_3d_info), axis=0)
            registered_3d_points_repro_error[tag_id] = np.concatenate((registered_3d_points_repro_error[tag_id], corresponding_3d_repro_error), axis=0)


# with open(r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\room1\apriltags_repro_error.json", "w") as outfile:
#    json.dump({k: v.tolist() for k, v in registered_3d_points_repro_error.items()}, outfile, indent=4)



There are totally 1 tags in the frame
There are totally 1 tags in the frame


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 2 tags in the frame
There are totally 2 tags in the frame
There are to

There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 2 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 1 tags in the frame
There are totally 2 tags in the frame


In [9]:
filtered_registered_tag_points_xyz, outliers_xyz = filter_registered_points(registered_3d_points_xyz, registered_3d_points_repro_error)

keypoints_rc = []
for points in filtered_registered_tag_points_xyz.values():
    keypoints_rc.append(np.mean(points, axis = 0))
keypoints_rc = np.array(keypoints_rc)

0.6159682487356248
[0] [10] 3


In [151]:
# keypoints_gt = np.array(a1d1)
# dist_gt = np.linalg.norm(keypoints_gt[-1] - keypoints_gt[1])
# dist_rc = np.linalg.norm(keypoints_rc[0] - keypoints_rc[1])

# print(dist_gt/dist_rc)

0.5292238029690113


In [127]:
pcd_copy = copy.deepcopy(pcd_reconstruction)
pcd_copy.paint_uniform_color(np.array([220, 220 ,220])/255)


# for tag in registered_3d_points_info.keys():
#     for info in registered_3d_points_info[tag]:
#         print(info.error)
    #visualization_3d(pcd_rc, registered_3d_points_xyz, highlight=False)

pcd_outliers = create_pcd(outliers_xyz, color = [0, 0 ,0])
vis = [pcd_copy, pcd_outliers]
for i, tag_id in enumerate(filtered_registered_tag_points_xyz.keys()):

    pcd_tag = create_geometry_at_points(filtered_registered_tag_points_xyz[tag_id], colorbar[i], radius = 0.1)
    vis.append(pcd_tag)

o3d.visualization.draw_geometries(vis)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 


In [10]:
#keypoints_gt = np.flip(keypoints_gt, 0)
keypoints_gt = np.array(a1d1)
mesh_gt = o3d.io.read_triangle_mesh(mesh_fullpath)
num_points = 30000
pcd_gt = mesh_gt.sample_points_uniformly(num_points)

In [75]:
s_opt, R_opt,t_opt = rigid_transform_3D(np.asmatrix(keypoints_gt), np.asmatrix(keypoints_rc), True)


In [76]:
s_opt = s_opt /1
print(s_opt)

0.7044136361881836


In [153]:
# s_opt = 0.5292238029690113

In [130]:
# est_extrinsic = np.concatenate(
#                     [np.concatenate([R_opt, t_opt.reshape(3, 1)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

# pcd_gt= o3d.geometry.PointCloud()
# pcd_gt.points = o3d.utility.Vector3dVector(keypoints_gt)

# pcd_rc= o3d.geometry.PointCloud()
# pcd_rc.points = o3d.utility.Vector3dVector(keypoints_rc)

# pcd_rc.scale(s_opt ,center=np.zeros(3))

# draw_registration_result(pcd_rc, pcd_gt, est_extrinsic)


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 


In [77]:
Aligner = MeshAlignment(pcd_reconstruction, pcd_gt, 30000, scale_factor = s_opt, PCD = True, filtering = True,voxel_size=0.03, icp_method="multiscale")
Aligner.register()

Statistical oulier removal
Statistical oulier removal
Statistical oulier removal
:: Normalize point clouds
:: Downsample with a voxel size 0.030.
4758
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
:: Downsample with a voxel size 0.030.
4887
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.030,
   we use a liberal distance threshold 0.045.
3. Colored point cloud registration
[2000, 0.02, 0]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
[1000, 0.01, 1]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
[200, 0.005, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration


In [78]:
# We need to recorver the translation because the point cloud we used in aligner is normalized and prescaled.
# Rotation is not affected by those factors
delta_tran = Aligner.centroid_gt - Aligner.result_icp.transformation[:3, :3] @ (Aligner.centroid_rc * Aligner.scale_factor)
tran_new = Aligner.result_icp.transformation[:3, 3].reshape(3, 1) * Aligner.backward_factor + delta_tran.reshape(3, 1)
est_new = np.concatenate(
                    [np.concatenate([Aligner.result_icp.transformation[:3, :3], tran_new], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

In [79]:
pcd_reconstruction_vis = copy.deepcopy(pcd_reconstruction)
pcd_reconstruction_vis.scale(s_opt, center = np.zeros(3))
Aligner.draw_registration_result(pcd_reconstruction_vis, mesh_gt, est_new, colored=True, inverse=False)

In [82]:
PI_registration = {}
for i, image_id in enumerate(images):
    if i % 2!=0:
        continue
    image = images[image_id]
    
    rot = qvec2rotmat(image.qvec)
    colmap_world2cam = np.concatenate(
                    [np.concatenate([rot, image.tvec.reshape(3,1)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

    register_result = est_new @ np.linalg.inv(colmap_world2cam)
    register_result[:3, 3] = register_result[:3, 3] * s_opt
    

    PI_registration[image.name] = register_result


print(len(PI_registration.keys()))


207


In [84]:
with open(r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1/result_colmap.json", "w") as outfile:
    json.dump({k: v.tolist() for k, v in PI_registration.items()}, outfile, indent=4)

In [81]:
visualization_registration(PI_registration, mesh_gt)

Output()

In [86]:
with open(r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1/result_colmap.json", "r") as f:
    PI_registration = json.load(f)

In [90]:

mesh_trimesh = trimesh.load("/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1/data3d/textured_output.obj")
im = Image.open(r"/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1/data3d/textured_output.jpg")
#tex = trimesh.visual.TextureVisuals(image=im)
#print(mesh_trimesh.visual.uv)
#mesh_trimesh.visual.texture = tex
color = trimesh.visual.uv_to_color(mesh_trimesh.visual.uv, im)
mesh_trimesh.visual.color = color

camera_pose = np.array(PI_registration['00011.jpg'])
mesh = pyrender.Mesh.from_trimesh(mesh_trimesh)
scene = pyrender.Scene()
scene.add(mesh)
# pyrender.Viewer(scene, use_raymond_lighting=True)


camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
s = np.sqrt(2)/2


PI_intrinsics = np.array([[766.2927454396544, 0.0, 543.6272327745995],
                                [0.0, 766.3976103393867, 566.0580149497666],
                                [0.0, 0.0, 1.0]])

img_width = 1088
img_height = 1080

scene.add(camera, pose=camera_pose)
light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
scene.add(light, pose=camera_pose)
r = pyrender.OffscreenRenderer(img_width, img_height)
color, depth = r.render(scene)
# plt.figure()
# plt.subplot(1,2,1)
# plt.axis('off')
plt.imshow(color)
plt.axis('off')
plt.show()

XXX: resource creation failed
XXX: resource creation failed


GLError: GLError(
	err = 1285,
	description = b'out of memory',
	baseOperation = glGenerateMipmap,
	cArguments = (GL_TEXTURE_2D,)
)

In [180]:
IPhone_registration = {}
IPhone_GT = {}
pose_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1\pose"
r = R.from_euler('xyz', [0, 180, 180], degrees=True)
Additional_Rotation = r.as_matrix()

additional_rotation = np.concatenate(
                [np.concatenate([Additional_Rotation, np.zeros((3,1))], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

for i, image_id in enumerate(images):
    # if i % 20!=0:
    #     continue
    image = images[image_id]
    json_name = image.name.replace('jpg', 'json')

    with open(os.path.join(pose_path, json_name), 'r') as f:
        camera_param = json.load(f)

    ext_gt =  np.array(camera_param["cameraPoseARFrame"]).reshape(4, 4)  @ additional_rotation

    
    rot = qvec2rotmat(image.qvec)
    colmap_world2cam = np.concatenate(
                    [np.concatenate([rot, image.tvec.reshape(3,1)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

    register_result = est_new @ np.linalg.inv(colmap_world2cam)
    register_result[:3, 3] = register_result[:3, 3] * s_opt
    

    IPhone_registration[image.name] = register_result
    IPhone_GT[image.name] = ext_gt


print(len(IPhone_registration.keys()))
print(len(IPhone_GT.keys()))

117
117


In [183]:
visualization_PI_IPhone(IPhone_GT, IPhone_registration ,  mesh_gt)

d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [49]:
PI_registration["01321.jpg"]

array([[-0.91805858, -0.3806811 ,  0.11068129,  0.21480709],
       [ 0.35735638, -0.9155172 , -0.18472863, -0.50719932],
       [ 0.17165333, -0.13003904,  0.97653724, -3.05254395],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [181]:
pos_error = []
rot_error = []
for gt_pose, est_pose in zip(IPhone_registration.values(), IPhone_GT.values()):
    pos_error.append(position_error(est_pose[:3, 3], gt_pose[:3, 3]))
    rot_error.append(rotation_error(est_pose[:3, :3], gt_pose[:3, :3]))


In [184]:
print(pos_error)

print(sum(pos_error)/len(pos_error))

[0.1168391553520796, 0.11105689632457016, 0.10565543176618353, 0.11369112382740497, 0.10973845592075394, 0.09727571279680233, 0.09632918527710187, 0.12549694224545482, 0.10379231045965384, 0.1152914080353012, 0.10368661493530491, 0.08762490125973327, 0.12574400416145629, 0.08486749591815809, 0.02945901380954329, 0.07123032770907399, 0.1053684599047804, 0.07434248816256814, 0.028237349526207357, 0.06728009910574874, 0.08162667907282056, 0.052612637376784725, 0.07245569065933723, 0.052166608203055874, 0.05834690315983716, 0.05292690590008789, 0.03734074723802894, 0.04797835684498888, 0.06192995016585236, 0.035727758894619766, 0.052482431374834715, 0.027066631222303125, 0.04726231406153163, 0.03330210861562493, 0.02922191260804831, 0.0326979449550595, 0.018446319331848576, 0.039204924760694176, 0.022668713806235325, 0.019333043617062867, 0.036084522467467114, 0.03182772003618028, 0.022105030179333674, 0.040191005565892154, 0.023956478670535485, 0.02572653729499393, 0.030110184994963603, 0

In [182]:
print(rot_error)

print(sum(rot_error)/len(rot_error))

[1.3374520857634875, 1.2694635075395293, 1.2306742123064238, 1.1814288668932114, 0.9739221112876549, 0.8437684607268737, 0.8905598494551664, 1.4879086043413745, 1.0175707541435963, 1.3316623910505876, 1.0758992757172001, 1.0256813925296093, 1.4802466805307068, 0.9986141810823644, 0.5820482121886773, 0.8195736973858024, 1.0806178078285689, 0.9160759366559211, 0.5350723590474957, 0.910410148051961, 0.8977712516439592, 0.8011129983822765, 0.9864315688187442, 0.854974734018214, 0.8247284227633513, 0.7982917489687534, 0.4255038954028892, 0.7566997473190137, 0.7980817000259575, 0.7537491841440337, 0.7350109783757549, 0.6615146368759273, 0.7931336259780188, 0.5948484383363551, 0.692715545026475, 0.6732437198514332, 0.5927274781943047, 0.4330349050609745, 0.6235147980590605, 0.6183444442003915, 0.6104135072955692, 0.5902298693787151, 0.5537264364928144, 0.3178772520901705, 0.5749486875504096, 0.5841698688957861, 0.6226397824333098, 0.48980304527255697, 1.060608435724376, 0.4047784740158338, 0.